# LSTM (Long Short Term Memory) for title generation based on YouTube blogger's chanel

In [1]:
import re
from pytube import YouTube
from pytube import Playlist
from pytube import Channel

# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
# from tensorflow import set_random_seed
from numpy.random import seed
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Init Plugin
Init Graph Optimizer
Init Kernel


In [103]:
# all_headlines = []

# playlist = Playlist("https://www.youtube.com/watch?v=6I2L7qORwN4&list=PLVqpOPG2Lex_htBmJFI0SMfU44S6msx5r")
# playlist._video_regex = re.compile(r"\"url\":\"(/watch\?v=[\w-]*)")
# print('Number of videos in playlist: %s' % len(playlist.video_urls))
# for url in playlist.video_urls:
#     yt = YouTube(url)
#     all_headlines.append(yt.title[:-29])

# all_headlines[:10]

In [3]:
c = Channel('https://www.youtube.com/c/maxkatz1')
print(f'Downloading videos by: {c.channel_name}')
print('Всего видео на канале: ', len(c.videos))
all_headlines = []
for video in c.videos:
    all_headlines.append(video.title)
all_headlines[:7]

Всего видео на канале:  721


['Доллар дешевеет, цены растут. Что будет дальше с Россией? (English subs) / @Максим Кац',
 'Бурятия, Дагестан и другие регионы на убой. Чьими руками воюет Путин (English subs) / @Максим Кац',
 'Как правильно дезактивировать пропаганду. Мнение (English subs) / @Максим Кац',
 'Путин уничтожает Россию. Не будьте частью этого (English subs) / @Максим Кац',
 'Чего добился Путин. Итоги войны России против Украины (English subs) / @Максим Кац',
 'Противостояние Путина с НАТО (English subs) / @Максим Кац',
 'Война 2022. Когда всё началось (English subs) / @Максим Кац']

In [4]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    return txt 
    
corpus = [clean_text(x) for x in all_headlines]
corpus[:7]

['доллар дешевеет цены растут что будет дальше с россией english subs  максим кац',
 'бурятия дагестан и другие регионы на убой чьими руками воюет путин english subs  максим кац',
 'как правильно дезактивировать пропаганду мнение english subs  максим кац',
 'путин уничтожает россию не будьте частью этого english subs  максим кац',
 'чего добился путин итоги войны россии против украины english subs  максим кац',
 'противостояние путина с нато english subs  максим кац',
 'война 2022 когда всё началось english subs  максим кац']

In [5]:
# Tokenization is a process of extracting tokens (terms / words) from a corpus. 
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words


# In the above output [30, 507], [30, 507, 11], [30, 507, 11, 1] and so on 
# represents the ngram phrases generated from the input data. 
# every integer corresponds to the index of a particular word in the complete vocabulary of words present in the text. 
# For example
inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[583, 584],
 [583, 584, 195],
 [583, 584, 195, 301],
 [583, 584, 195, 301, 7],
 [583, 584, 195, 301, 7, 47],
 [583, 584, 195, 301, 7, 47, 52],
 [583, 584, 195, 301, 7, 47, 52, 6],
 [583, 584, 195, 301, 7, 47, 52, 6, 196],
 [583, 584, 195, 301, 7, 47, 52, 6, 196, 10],
 [583, 584, 195, 301, 7, 47, 52, 6, 196, 10, 11]]

In [6]:
from keras.utils import np_utils

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.np_utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [7]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-22 17:31:08.747676: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-22 17:31:08.748985: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 10)            25180     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2518)              254318    
Total params: 323,898
Trainable params: 323,898
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(predictors, label, epochs=150, verbose=5)

Epoch 1/150
Epoch 2/150
Epoch 3/150
Epoch 4/150
Epoch 5/150
Epoch 6/150
Epoch 7/150
Epoch 8/150
Epoch 9/150
Epoch 10/150
Epoch 11/150
Epoch 12/150
Epoch 13/150
Epoch 14/150
Epoch 15/150
Epoch 16/150
Epoch 17/150
Epoch 18/150
Epoch 19/150
Epoch 20/150
Epoch 21/150
Epoch 22/150
Epoch 23/150
Epoch 24/150
Epoch 25/150
Epoch 26/150
Epoch 27/150
Epoch 28/150
Epoch 29/150
Epoch 30/150
Epoch 31/150
Epoch 32/150
Epoch 33/150
Epoch 34/150
Epoch 35/150
Epoch 36/150
Epoch 37/150
Epoch 38/150
Epoch 39/150
Epoch 40/150
Epoch 41/150
Epoch 42/150
Epoch 43/150
Epoch 44/150
Epoch 45/150
Epoch 46/150
Epoch 47/150
Epoch 48/150
Epoch 49/150
Epoch 50/150
Epoch 51/150
Epoch 52/150
Epoch 53/150
Epoch 54/150
Epoch 55/150
Epoch 56/150
Epoch 57/150
Epoch 58/150
Epoch 59/150
Epoch 60/150
Epoch 61/150
Epoch 62/150
Epoch 63/150
Epoch 64/150
Epoch 65/150
Epoch 66/150
Epoch 67/150
Epoch 68/150
Epoch 69/150
Epoch 70/150
Epoch 71/150
Epoch 72/150
Epoch 73/150
Epoch 74/150
Epoch 75/150
Epoch 76/150
Epoch 77/150
Epoch 78

In [111]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [185]:
print (generate_text("Без Путина", 13, model, max_sequence_len))
print (generate_text("что будет дальше", 15, model, max_sequence_len))
print (generate_text("Противостояние Путина с", 13, model, max_sequence_len))
print (generate_text("Навального", 14, model, max_sequence_len))
print (generate_text("Может ли", 15, model, max_sequence_len).split()[-1])
print (generate_text("Что будет", 15, model, max_sequence_len))

Без Путина И России — В Тельавива Имперских Без Надо English Subs Максим Кац ​
Что Будет Дальше Россия И России — И Больше Имперских Замашек Надо Не Надо English Subs Максим Кац
Противостояние Путина С Нато Максим Кац Навальный Максим Кац Из Максим Кац Августа Манижа И Россия
Навального И Россия И Россия — России В Эфире Тельавива Где English Subs Максим Кац
Кац
Что Будет И Россия — России — И Больше Имперских Без Без English Subs Максим Кац ​


In [176]:
seed_w = set()
for h in corpus:
    seed_w.add(h.split()[0])
seed_w = list(seed_w)
print(seed_w[:10])

['форд', 'джордж', 'куда', 'палестиноизраильский', 'пражская', 'боятся', 'прогресс', 'смерть', 'израиль', 'маргарет']


In [203]:
HeadLines = []
for w in seed_w:
    h = generate_text(w, max_sequence_len, model, max_sequence_len)
    if 'English Subs Максим Кац' in h:
        pos = h.split().index('Кац')
        t = str()
        for hh in h.split()[:pos+1] : t+=(hh+' ')
        HeadLines.append(t)


In [204]:
len(HeadLines)

351

In [207]:
f = open('best_headings.txt', 'w')
for h in HeadLines:
    f.write(h+'\n')
f.close()

## Some results 

Ссср Как Как Не Надо English Subs Максим Кац 

Диктатура И Россия — России — И Больше Имперских Без Без English Subs Максим Кац

Хрущевки И Вашем — Россия И России С English Subs Максим Кац 

Многоэтажки И России — И Больше Имперских Замашек Без Не Надо English Subs Максим Кац

Голосовать И Россия И Россия — России В Эфире Тельавива Где English Subs Максим Кац

Очереди И Россия — России — И Больше Имперских Без Без Не English Subs Максим Кац 

Мир И Вашем — Россия И России С English Subs Максим Кац 

Образование И России — И Больше Имперских Замашек Надо Не Надо English Subs Максим Кац 

Иллюзия Как Как Как Не Надо English Subs Максим Кац 

Разоблачения Как Как Как Не Надо English Subs Максим Кац 

Народный Не Надо English Subs Максим Кац 

Ложь И Россия — России Как English Subs Максим Кац 

Отрицательный Как Как Как Не Надо English Subs Максим Кац 

Российские И Россия — России Как English Subs Максим Кац 

Саакашвили Не Надо English Subs Максим Кац 

Евросоюз И России Как English Subs Максим Кац 

Ужасы И Россия — России Как English Subs Максим Кац 

​Максим И Россия — России Как English Subs Максим Кац 

Буча Как Как Как Не Надо English Subs Максим Кац 

Революция И Россия — России Как English Subs Максим Кац 